In [1]:
import pandas as pd 
import sqlite3


In [3]:
#connecting to DB
con = sqlite3.connect("msgstore.db")


In [6]:
"""
The code creates a view called "message_data" by joining multiple tables together
and selecting specific columns from those tables.
It also includes a CASE statement to determine the message type 
and a WHERE clause to filter out irrelevant rows.
This view can be used to easily query and work with data from the WhatsApp chat backup,
making it more manageable for analysis.
"""
con.execute("""
            CREATE VIEW message_data
            AS
              SELECT mssg._id,
                     mssg.from_me,
                     CASE mssg.message_type
                       WHEN 0 THEN 'text'
                       WHEN 1 THEN 'image'
                       WHEN 2 THEN 'audio'
                       WHEN 3 THEN 'video'
                       WHEN 4 THEN 'contact'
                       WHEN 5 THEN 'location'
                       WHEN 9 THEN 'document'
                       WHEN 13 THEN 'gif'
                       WHEN 20 THEN 'sticker'
                       ELSE 'unknown'
                     END                                                                 AS
                        message_type,
                     Strftime('%Y-%m-%d %H:%M:%S', mssg.timestamp / 1000.0, 'unixepoch') AS
                        timestamp,
                     mssg.text_data,
                     mssg.starred,
                     jid.USER                                                            AS
                        chat_number,
                     jid1.USER                                                           AS
                        sender,
                     moar.reaction,
                     mssg_fd.forward_score,
                     mssg_lnk.link_index,
                     mssg_md.file_path,
                     mssg_md.mime_type,
                     media_duration,
                     page_count,
                     mssg_mnt.message_row_id                                             AS
                        mention_msg_id,
                     subject
              FROM   message AS mssg
                     LEFT JOIN chat
                            ON mssg.chat_row_id = chat._id
                     LEFT JOIN jid
                            ON chat.jid_row_id = jid._id
                     LEFT JOIN jid AS jid1
                            ON mssg.sender_jid_row_id = jid1._id
                     LEFT JOIN message_add_on AS mao
                            ON mssg._id = mao.parent_message_row_id
                     LEFT JOIN message_add_on_reaction AS moar
                            ON mao._id = moar.message_add_on_row_id
                     LEFT JOIN message_add_on_receipt_device AS moad
                            ON mao._id = moad.message_add_on_row_id
                     LEFT JOIN message_forwarded AS mssg_fd
                            ON mssg._id = mssg_fd.message_row_id
                     LEFT JOIN message_link AS mssg_lnk
                            ON mssg._id = mssg_lnk.message_row_id
                     LEFT JOIN message_media AS mssg_md
                            ON mssg._id = mssg_md.message_row_id
                     LEFT JOIN message_mentions AS mssg_mnt
                            ON mssg._id = mssg_mnt.message_row_id
              WHERE  jid.USER <> 'status' 
              """);

In [9]:
"""
This code retrieves the count of different media types (image, audio, sticker, document, video, gif)
in the "message_data" view 
"""
pd.read_sql("""
            SELECT Upper(message_type) AS media_type,
                   Count(message_type) AS count
            FROM   message_data
            WHERE  message_type IN ( 'image', 'audio', 'sticker', 'document',
                                     'video', 'gif' )
            GROUP  BY message_type
            ORDER  BY count DESC 
            """,con)



,media_type,count
0,IMAGE,1356
1,STICKER,216
2,VIDEO,66
3,DOCUMENT,55
4,AUDIO,24
5,GIF,5


In [11]:
"""
Same as the previous but for each chat" 
"""
pd.read_sql("""
            SELECT chat_number,
                   Sum(CASE
                         WHEN message_type IN ( 'image', 'audio', 'sticker', 'document',
                                                'video', 'gif' ) THEN 1
                         ELSE 0
                       end) AS Total,
                   Sum(CASE
                         WHEN message_type = 'image' THEN 1
                         ELSE 0
                       end) AS Image,
                   Sum(CASE
                         WHEN message_type = 'audio' THEN 1
                         ELSE 0
                       end) AS Audio,
                   Sum(CASE
                         WHEN message_type = 'sticker' THEN 1
                         ELSE 0
                       end) AS Sticker,
                   Sum(CASE
                         WHEN message_type = 'document' THEN 1
                         ELSE 0
                       end) AS Document,
                   Sum(CASE
                         WHEN message_type = 'video' THEN 1
                         ELSE 0
                       end) AS Video,
                   Sum(CASE
                         WHEN message_type = 'gif' THEN 1
                         ELSE 0
                       end) AS Gif
            FROM   message_data
            WHERE  subject IS NULL
            GROUP  BY chat_number
            ORDER  BY total DESC
            LIMIT  10 
            """,con)



,chat_number,Total,Image,Audio,Sticker,Document,Video,Gif
0,640,484,157,157,157,0,10,3
1,412,250,36,3,195,7,6,3
2,852,197,13,140,42,2,0,0
3,462,152,85,3,56,4,3,1
4,980,143,42,53,45,0,3,0
5,961,117,42,25,45,0,5,0
6,274,106,36,7,50,8,5,0
7,115,95,71,22,2,0,0,0
8,888,95,10,85,0,0,0,0
9,988,81,9,57,10,4,1,0


In [12]:
#top 10 most common reaction
pd.read_sql("""
            SELECT reaction,
                   Count(reaction) AS count
            FROM   message_data
            WHERE  reaction <> ''
            GROUP  BY reaction
            ORDER  BY count DESC
            LIMIT  10 
            """,con)


,reaction,count
0,❤️,1498
1,😂,275
2,😢,106
3,👍🏻,38
4,💖,32
5,😮,17
6,👍,10
7,🙏,7
8,😘,4
9,❤,0


In [13]:
# top 10 most common reactions across chats
pd.read_sql("""
            SELECT chat_number,
                   Count(reaction) AS Total,
                   Sum (CASE
                          WHEN reaction = '❤️' THEN 1
                          ELSE 0
                        end)       AS ❤️,
                   Sum (CASE
                          WHEN reaction = '😂' THEN 1
                          ELSE 0
                        end)       AS 😂,
                   Sum (CASE
                          WHEN reaction = '😢' THEN 1
                          ELSE 0
                        end)       AS 😢,
                   Sum (CASE
                          WHEN reaction = '😮' THEN 1
                          ELSE 0
                        end)       AS 😮,
                   Sum (CASE
                          WHEN reaction = '👍' THEN 1
                          ELSE 0
                        end)       AS 👍,
                   Sum (CASE
                          WHEN reaction NOT IN ( '❤️', '😂', '👍', '😢', '😮' )
                        THEN 1
                          ELSE 0
                        end)       AS OTHER
            FROM   message_data
            WHERE  subject IS NULL
            GROUP  BY chat_number
            ORDER  BY total DESC
            LIMIT  10 
            """,con)


,chat_number,Total,❤️,😂,😢,😮,👍,OTHER
0,412,426,370,42,7,0,3,4
1,640,115,55,1,33,1,0,25
2,274,71,33,5,27,1,0,5
3,980,71,46,16,7,0,0,2
4,852,57,56,0,0,0,0,1
5,988,24,24,0,0,0,0,0
6,961,18,10,5,3,0,0,0
7,462,16,12,2,1,0,0,1
8,115,15,8,5,2,0,0,0
9,888,7,7,0,0,0,0,0


In [14]:
# top 10 chat numbers with the most number of starred messages and the count of starred messages for each chat
pd.read_sql("""
            SELECT chat_number,
                   Sum(starred) AS starred_count
            FROM   message_data
            WHERE  subject IS NULL
            GROUP  BY chat_number
            HAVING Sum(starred) > 0
            ORDER  BY starred_count DESC
            LIMIT  10 
            """,con)


,chat_number,starred_count
0,412,347
1,640,81
2,274,59
3,852,27
4,980,23
5,462,11
6,988,10
7,115,6
8,888,2
9,961,0


In [15]:
"""
top 10 chat numbers with the most audio duration, showing the total duration of audio messages sent by the user (ME),
received from friends (Friends) and the total audio duration for each chat
"""
pd.read_sql("""
            SELECT chat_number,
                   Round (Sum(CASE
                                WHEN from_me = 1 THEN media_duration
                                ELSE 0
                              end) / 60., 2)            AS ME,
                   Round (Sum(CASE
                                WHEN from_me = 0 THEN media_duration
                                ELSE 0
                              end) / 60., 2)            AS Friends,
                   Round (Sum(media_duration) / 60., 2) AS Total
            FROM   message_data
            WHERE  mime_type LIKE ( 'audio%' )
                   AND subject IS NULL
            GROUP  BY chat_number
            ORDER  BY total DESC
            LIMIT  10 
            """,con)


,chat_number,ME,Friends,Total
0,640,4.60,68.26,72.86
1,274,32.52,14.80,47.32
2,852,16.31,24.66,40.97
3,980,19.00,21.76,40.76
4,412,14.20,13.67,27.87
5,462,8.59,6.46,15.05
6,961,0.63,11.04,11.67
7,988,5.74,1.64,7.38
8,888,2.72,2.99,5.71
9,115,3.66,0.54,4.20


In [16]:
# top 10 most common messages and their occurrence
pd.read_sql("""
            SELECT text_data        AS message,
                   Count(text_data) AS occurrence
            FROM   message_data
            WHERE  text_data <> ''
            GROUP  BY text_data
            ORDER  BY occurrence DESC
            LIMIT  10 
            """,con)


,message,occurrence
0,😂😂😂,232
1,😂😂,167
2,ماشي,103
3,😂😂😂😂😂,89
4,لسة,71
5,😂😂😂😂,63
6,خير,55
7,لاء,49
8,تمام,37
9,يارب,13


In [17]:
#Most used emojis across Top 5 chats per sender
pd.read_sql(""" 
            WITH emojis AS
            (
                     SELECT   chat_number,
                              CASE
                                       WHEN from_me=1 THEN 'ME'
                                       ELSE 'FRIEND'
                              END                                                                     AS sender ,
                              Sum(Length(text_data)-Length(Replace(text_data,'😂','')))/Length('😂') AS 😂,
                              Sum(Length(text_data)-Length(Replace(text_data,'🤭','')))/Length('🤭') AS 🤭,
                              Sum(Length(text_data)-Length(Replace(text_data,'😭','')))/Length('😭') AS 😭,
                              Sum(Length(text_data)-Length(Replace(text_data,'💔','')))/Length('💔') AS 💔,
                              Sum((Length(text_data)-Length(Replace(text_data,'❤','')))/Length('❤')+ 
                                  (Length(text_data)-Length(Replace(text_data,'♥','')))/Length('♥'))  AS ❤,
                                  
                              Sum(Length(text_data)-Length(Replace(text_data,'🙈','')))/Length('🙈') AS 🙈,
                              Sum(Length(text_data)-Length(Replace(text_data,'🙂','')))/Length('🙂') AS 🙂,
                              Sum(Length(text_data)-Length(Replace(text_data,'😘','')))/Length('😘') AS 😘,
                              Sum(Length(text_data)-Length(Replace(text_data,'😢','')))/Length('😢') AS 😢
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY chat_number,
                              2), add_total AS
            (
                   SELECT     *,
                          😂+🤭+😭+💔+❤+🙈+🙂+😘+😢 AS sender_total
                   FROM   emojis),rank AS
            (
                     SELECT   chat_number,
                              Sum(sender_total)                                  AS chat_total,
                              Row_number() OVER(ORDER BY Sum(sender_total) DESC) AS rank
                     FROM     add_total
                     GROUP BY chat_number)
            SELECT     a.*,
                       r.chat_total
            FROM       add_total AS a
            INNER JOIN rank      AS r
            ON         a.chat_number=r.chat_number
            ORDER BY   rank limit 10
            """,con)



,chat_number,Sender,😂,🤭,😭,💔,❤,🙈,🙂,😘,😢,sender_total,chat_total
0,640,FRIEND,532,0,43,0,203,0,0,22,63,800,1921
1,640,ME,933,0,13,16,143,0,7,9,8,1121,1921
2,412,FRIEND,352,21,1,14,16,10,6,0,0,420,1214
3,412,ME,750,5,2,12,14,2,9,0,10,794,1214
4,980,FRIEND,289,0,0,0,43,0,0,0,38,332,525
5,980,ME,176,1,0,1,1,1,2,11,52,193,525
6,852,FRIEND,137,0,0,0,0,3,2,0,0,142,464
7,852,ME,253,57,0,0,2,10,0,0,24,322,464
8,274,FRIEND,170,0,4,0,1,0,6,0,7,181,241
9,274,ME,36,8,0,0,6,2,7,1,9,60,241


In [18]:
#total messages
con.execute("""
            SELECT Count(DISTINCT _id)
            FROM   message_data 
            """).fetchone()


(25762,)

In [19]:
#Top chats

con.execute("""
            SELECT chat_number
            FROM   message_data
            GROUP  BY chat_number
            ORDER  BY Count(_id) DESC
            LIMIT  5 
            """).fetchall()


[('644',), ('863',), ('666',), ('39',), ('264',)]

In [20]:
#first ever chat
con.execute("""
            SELECT chat_number
            FROM   message_data
            WHERE  subject IS NULL
            ORDER  BY timestamp
            LIMIT  1 
            """).fetchall()


[('80',)]

In [21]:
#longest 5 messages
con.execute("""
            SELECT timestamp,
                   from_me,
                   chat_number,
                   text_data,
                   Length(text_data) AS text_length
            FROM   (SELECT *
                    FROM   message_data
                    WHERE  _id IN (SELECT DISTINCT _id
                                   FROM   message_data))
            WHERE  subject IS NULL
            ORDER  BY text_length DESC
            LIMIT  5 
            """).fetchall();

In [22]:
#longest daily chatting streaks
pd.read_sql(""" 
            WITH daily AS
            (
                     SELECT   chat_number,
                              Strftime('%Y-%m-%d',timestamp) AS timestamp,
                              Count(_id)                     AS cnt
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY 1,
                              2), daily_next AS
            (
                   SELECT chat_number,
                          timestamp,
                          cnt,
                          CASE
                                 WHEN Julianday(Lead(timestamp) OVER(partition BY chat_number)) - Julianday(timestamp) =1 THEN 0
                                 ELSE 1
                          END AS next
                   FROM   daily), daily_sum AS
            (
                     SELECT   chat_number,
                              timestamp,
                              cnt,
                              next,
                              sum(next) OVER(partition BY chat_number ORDER BY chat_number,timestamp) AS sm
                     FROM     daily_next)
            SELECT   chat_number,
                     count(sm)      AS n_days,
                     min(timestamp) AS start,
                     max(timestamp) AS
                        END,
            sum(cnt) AS n_messages
            FROM     daily_sum
            GROUP BY chat_number,
                     sm
            ORDER BY 2 DESC limit 10
            """,con)


,chat_number,n_days,start,end,n_messages
0,412,22,2021-06-25,2021-07-18,389
1,640,17,2021-06-09,2021-06-27,536
2,274,13,2020-11-03,2020-11-16,79
3,980,12,2020-05-23,2020-06-04,37
4,852,10,2021-08-07,2021-08-17,87
5,462,10,2020-12-06,2020-12-18,69
6,961,10,2021-05-13,2021-05-23,26
7,115,10,2021-07-16,2021-07-27,95
8,988,8,2021-08-26,2021-09-03,363
9,888,8,2020-12-06,2020-12-14,50


In [23]:
# top 10 chat numbers with the most number of messages
pd.read_sql("""
            SELECT chat_number,
                   Count(_id) AS count
            FROM   message_data
            WHERE  subject IS NULL
            GROUP  BY chat_number
            ORDER  BY 2 DESC
            LIMIT  10
            """,con)


,chat_number,count
0,640,5043
1,980,4881
2,412,3175
3,961,2747
4,852,1906
5,274,1043
6,462,970
7,888,883
8,115,211
9,988,175


In [24]:
# top 10 chats with the most number of messages including total for each sender
pd.read_sql("""
            WITH rank AS
            (
                     SELECT   chat_number,
                              Count(_id)                                   AS n_messages,
                              Row_number() OVER (ORDER BY Count(_id) DESC) AS rank
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY chat_number )
            SELECT   m.chat_number,
                     Sum(
                     CASE
                              WHEN m.from_me=1 THEN 1
                              ELSE 0
                     END )AS me,
                     Sum(
                     CASE
                              WHEN m.from_me=0 THEN 1
                              ELSE 0
                     END )AS friends
            FROM     message_data m
            JOIN     rank r
            ON       m.chat_number = r.chat_number
            GROUP BY m.chat_number
            ORDER BY r.rank limit 10
            """,con)


,chat_number,ME,FRIENDS
0,640,2638,2404
1,980,2071,2809
2,412,1762,1412
3,961,1389,1358
4,852,1094,812
5,274,560,482
6,462,454,428
7,888,426,544
8,115,96,78
9,988,89,122


In [25]:
#MORE EFFICIENT IMPLEMENTATION
pd.read_sql("""
            WITH rank AS
            (
                     SELECT   chat_number,
                              Count(_id)                                   AS n_messages,
                              Row_number() OVER (ORDER BY Count(_id) DESC) AS rank
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY chat_number ),n_messages AS
            (
                     SELECT   chat_number,
                              CASE
                                       WHEN from_me = 1 THEN 'ME'
                                       WHEN from_me = 0 THEN 'OTHERS'
                              END        AS sender,
                              Count(_id) AS cnt
                     FROM     message_data
                     GROUP BY chat_number,
                              2 )
            SELECT   m.chat_number,
                     Sum(
                     CASE
                              WHEN sender='ME' THEN cnt
                     END )AS me,
                     Sum(
                     CASE
                              WHEN sender='OTHERS' THEN cnt
                     END)       AS others
            FROM     n_messages AS m
            JOIN     rank r
            ON       m.chat_number = r.chat_number
            GROUP BY m.chat_number
            ORDER BY r.rank limit 10
            """,con);

In [26]:

#total messages for the last  20 months
pd.read_sql("""
            WITH total_rows AS
            (
                   SELECT Count(DISTINCT Strftime('%Y-%m',timestamp)) AS total_rows
                   FROM   message_data )
            SELECT   Strftime('%Y-%m',timestamp) AS timestamp,
                     Count(_id)                  AS n_messages
            FROM     message_data
            GROUP BY 1
            ORDER BY timestamp ASC limit 20 offset
                     (
                            SELECT (total_rows - 20)
                            FROM   total_rows);
            """,con)


,timestamp,n_messages
0,2021-06,699
1,2021-07,724
2,2021-08,222
3,2021-09,954
4,2021-10,1690
5,2021-11,371
6,2021-12,1870
7,2022-01,173
8,2022-02,101
9,2022-03,231


In [28]:
#cumulative count of messages over months
pd.read_sql("""
            SELECT Strftime('%Y-%m', timestamp) AS timestamp,
                   Sum(Count(DISTINCT _id))
                     OVER (
                       ORDER BY timestamp)      AS n_messages
            FROM   message_data
            GROUP  BY 1 
            """,con)


,timestamp,n_messages
0,2018-03,0
1,2018-09,1
2,2018-11,1
3,2019-02,2
4,2019-03,3
5,2019-04,3
6,2019-06,3
7,2019-07,4
8,2019-09,5
9,2019-10,5


In [29]:
#total messages over time per sender

pd.read_sql("""
            WITH long
                 AS (SELECT Strftime('%Y-%m', timestamp) AS timestamp,
                            CASE
                              WHEN from_me = 1 THEN 'ME'
                              WHEN from_me = 0 THEN 'OTHERS'
                            END                          AS SENDER,
                            Count(_id)                   AS n_messages
                     FROM   message_data
                     GROUP  BY 1,
                               2)
            SELECT timestamp,
                   Sum(CASE
                         WHEN sender = 'ME' THEN n_messages
                       END) AS ME,
                   Sum(CASE
                         WHEN sender = 'OTHERS' THEN n_messages
                       END) AS OTHERS
            FROM   long
            GROUP  BY timestamp 
            """,con);

In [30]:
#total messages over time per chat

result=con.execute("""
                    WITH columns AS
                    (
                             SELECT   chat_number
                             FROM     message_data
                             GROUP BY chat_number
                             ORDER BY Count(_id) DESC limit 5 ), query AS
                    (
                           SELECT 'SELECT timestamp, '
                                         || Group_concat( 'SUM(CASE WHEN chat_number = '''
                                         || chat_number
                                         || ''' THEN n_messages ELSE 0 END) AS '''
                                         || chat_number
                                         || '''')
                                         || ' FROM pivot_table GROUP BY timestamp' AS query
                           FROM   columns )
                    SELECT query
                    FROM   query;
                    """)
query=result.fetchone()[0]
data=pd.read_sql(""" 
                WITH pivot_table AS
                (
                         SELECT   strftime('%Y-%m', timestamp) AS timestamp,
                                  chat_number,
                                  count(_id) AS n_messages
                         FROM     message_data
                         GROUP BY 1,
                                  2)
                  """+query,con)


,timestamp,644,863,666,39,264
0,2018-03,25,58,5,55,0
1,2018-09,61,41,103,0,0
2,2018-11,11,113,48,20,0
3,2019-02,95,100,117,100,0
4,2019-03,9,83,34,163,0
5,2019-04,116,258,75,0,0
6,2019-06,261,81,51,465,0
7,2019-07,96,11,9,167,0
8,2019-09,187,16,81,51,0
9,2019-10,105,46,218,64,0


In [31]:
"""
the highest number of messages sent in one chat for each day using a correlated subquery (Top Daily chat)
"""
pd.read_sql(""" 
            WITH daily AS
            (
                     SELECT   Strftime('%Y-%m-%d', timestamp) AS timestamp,
                              chat_number,
                              Count(_id) AS cnt
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY 1,
                              2 )
            SELECT   timestamp,
                     chat_number,
                     cnt
            FROM     daily AS OUTER
            WHERE    cnt=
                     (
                            SELECT Max(cnt)
                            FROM   daily AS INNER
                            WHERE  OUTER.timestamp=INNER.timestamp )
            ORDER BY cnt DESC limit 20
            """,con)


,timestamp,chat_number,cnt
0,2022-05-19,588,7
1,2022-04-19,588,467
2,2022-03-26,588,224
3,2022-03-29,588,29
4,2021-01-04,708,385
5,2022-05-04,588,292
6,2021-03-16,108,292
7,2022-06-16,108,235
8,2022-04-11,588,174
9,2021-05-04,738,224


In [32]:
#Top chat of each month
pd.read_sql("""
            WITH monthly_count AS
            (
                     SELECT   Strftime('%Y-%m', timestamp) AS timestamp,
                              chat_number,
                              Count(_id) AS cnt
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY 1,
                              2), max_monthly AS
            (
                     SELECT   timestamp AS month,
                              Max(cnt)  AS n_messages
                     FROM     monthly_count
                     GROUP BY timestamp )
            SELECT     month,
                       n_messages,
                       chat_number
            FROM       max_monthly   AS mx
            INNER JOIN monthly_count AS ct
            ON         mx.n_messages=ct.cnt
            ORDER BY   month DESC limit 10
            """,con)


,month,n_messages,chat_number
0,2023-01,1287,12
1,2022-12,1358,588
2,2022-11,1789,588
3,2022-10,1283,588
4,2022-09,103,708
5,2022-08,51,120
6,2022-07,214,120
7,2022-06,185,708
8,2022-05,1,708
9,2022-04,599,552


In [33]:
#the highest number of messages sent in a single day for each chat
pd.read_sql("""
            WITH daily_count AS
            (
                     SELECT   Strftime('%Y-%m-%d', timestamp) AS timestamp,
                              chat_number,
                              Count(_id) AS cnt
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY 1,
                              2)
            SELECT   chat_number ,
                     Max(cnt) AS n_messages
            FROM     daily_count
            GROUP BY chat_number
            ORDER BY n_messages DESC limit 10
            """,con)



,chat_number,n_messages
0,588,684
1,108,379
2,12,149
3,714,62
4,552,54
5,120,41
6,552,32
7,738,29
8,708,20
9,876,7


In [34]:
#the number of times a chat had the highest number of daily messages
pd.read_sql(""" 
            WITH daily AS
            (
                     SELECT   Strftime('%Y-%m-%d', timestamp) AS timestamp,
                              chat_number,
                              Count(_id) AS cnt
                     FROM     message_data
                     WHERE    subject IS NULL
                     GROUP BY 1,
                              2 ), daily_max AS
            (
                   SELECT timestamp,
                          chat_number,
                          cnt
                   FROM   daily AS OUTER
                   WHERE  cnt=
                          (
                                 SELECT Max(cnt)
                                 FROM   daily AS INNER
                                 WHERE  OUTER.timestamp=INNER.timestamp ))
            SELECT   chat_number,
                     Count(chat_number) AS cnt
            FROM     daily_max
            GROUP BY chat_number
            ORDER BY cnt DESC limit 10
            """,con)


,chat_number,cnt
0,552,72
1,552,42
2,708,30
3,120,16
4,12,16
5,108,16
6,108,11
7,588,11
8,768,2
9,738,2
